In [11]:
import random
import numpy 
from importlib import reload
import matplotlib.pyplot as plt
import networkx as nx

# real dataset
fb = nx.read_edgelist('data/sample-ch2017.txt')
no_node = nx.number_of_nodes(fb)
print (nx.info(fb))

Name: 
Type: Graph
Number of nodes: 31135
Number of edges: 642287
Average degree:  41.2582


In [2]:
# number of edges if the graph was complete : f(n) = (n*(n-1))/2
#cg = nx.complete_graph(no_node) # takes too long to execute thus hashed
#print(nx.number_of_nodes(cg))

fn = (no_node * (no_node - 1)) / 2
fn

484678545.0

In [3]:
# Local bridges (no common contacts)
lb = list(nx.local_bridges(fb , with_span = False))
print('Number of Local bridges in 2017', len(list(nx.local_bridges(fb, with_span = False))))

# To create a list of nodes that belong to local node. from [('a','b'), ('c','b')] to ['a','b','c','b']
new_lb = []
for tup in lb:
    for element in tup:
        new_lb.append(element)        
#nd is to remove duplicates
nd = list(set(new_lb))

Number of Local bridges in 2017 96038


In [12]:
# to find second degree contacts of a single node

def find_nodes(graph, node, distance):

     # get all nodes within distance around the query node
     nodes = set(nx.ego_graph(graph, node, radius=distance))

     # remove nodes that are not **at** the specified distance but closer: removes all the first degree contact
     if distance > 1:
      nodes -= set(nx.ego_graph(graph, node, radius=distance-1))

     return list(nodes)

# example implementation
ba = find_nodes(fb, nd[4], 2) # reeturns a list of all the nodes tat are 2nd deg
print((len(ba)))


1096


In [17]:
# second degree contacts that pass through the LB of a single node
# two parameters that it takes are 1) the graph itself here fb 2) one specific local bridge (pair of nodes)
# it returns : predictive edges between the [second degree node, the node that is the part of LB]  in that same order
# returned variable is a nested list with the order 2
def seconddegree (fb, llbb):
    #llb = list(lb[0])
    b = llbb[0]
    win = []
    n = list(fb.neighbors(llbb[1]))
    pair =[]
    
    for a in range(len(n)):
        lis = []
        lis.append(n[a])
        lis.append(b)
        win.append(lis)
    return win

# example implementation
vr = seconddegree(fb, lb[0])
print(vr)

[['7109699', '7394358'], ['7394358', '7394358'], ['6445187', '7394358'], ['15834620', '7394358'], ['12676456', '7394358'], ['3288614', '7394358'], ['8608401', '7394358'], ['3508642', '7394358'], ['4314068', '7394358'], ['6328848', '7394358'], ['11263739', '7394358'], ['7289901', '7394358'], ['19377693', '7394358'], ['17042143', '7394358'], ['20233629', '7394358'], ['10199686', '7394358'], ['8031066', '7394358'], ['14612556', '7394358'], ['23305655', '7394358'], ['13427664', '7394358'], ['17025', '7394358'], ['3552484', '7394358'], ['10154844', '7394358'], ['8619915', '7394358'], ['14526589', '7394358'], ['8819366', '7394358'], ['4319886', '7394358'], ['17047813', '7394358'], ['3206979', '7394358'], ['5661869', '7394358'], ['16126212', '7394358'], ['13779793', '7394358'], ['6378588', '7394358'], ['13652177', '7394358'], ['3398343', '7394358'], ['6415607', '7394358'], ['15187077', '7394358'], ['12684497', '7394358'], ['14993087', '7394358'], ['25815033', '7394358'], ['7732357', '7394358'

In [18]:
# works !! to get all the second degree edges with their respective nodes ! 
# It applies the above mentioned function to all the pair of nodes that form a local bridge
# returns nested list of order 3 . 
# non existing edges ! predicted false edges
# second node in this return set (pe) is always the node that is the part of a local bridge
def allnodesseconddegree (fb, lb):
    pe = []
    for a in range(len(lb)):
        var = seconddegree(fb, lb[a])
        pe.append(var)
    return pe

vv = allnodesseconddegree(fb, lb)
#vv

[['7109699', '7394358'],
 ['7394358', '7394358'],
 ['6445187', '7394358'],
 ['15834620', '7394358'],
 ['12676456', '7394358'],
 ['3288614', '7394358'],
 ['8608401', '7394358'],
 ['3508642', '7394358'],
 ['4314068', '7394358'],
 ['6328848', '7394358'],
 ['11263739', '7394358'],
 ['7289901', '7394358'],
 ['19377693', '7394358'],
 ['17042143', '7394358'],
 ['20233629', '7394358'],
 ['10199686', '7394358'],
 ['8031066', '7394358'],
 ['14612556', '7394358'],
 ['23305655', '7394358'],
 ['13427664', '7394358'],
 ['17025', '7394358'],
 ['3552484', '7394358'],
 ['10154844', '7394358'],
 ['8619915', '7394358'],
 ['14526589', '7394358'],
 ['8819366', '7394358'],
 ['4319886', '7394358'],
 ['17047813', '7394358'],
 ['3206979', '7394358'],
 ['5661869', '7394358'],
 ['16126212', '7394358'],
 ['13779793', '7394358'],
 ['6378588', '7394358'],
 ['13652177', '7394358'],
 ['3398343', '7394358'],
 ['6415607', '7394358'],
 ['15187077', '7394358'],
 ['12684497', '7394358'],
 ['14993087', '7394358'],
 ['25815

In [ ]:
# This works !!
#now for the last condition ! No other mutual friend apart from ***
# returns thee nested list of all the edges(node pair) that have only one mutual friend that is a part of LB
interseclist = []
ppe = []

for a in range(len(vv)):
    for b in range(len(vv[a])):
        cc = vv[a]
        #print(type(cc))
        #print(cc[0])
        
        for c in range(len(cc[b])):
            dd = cc[b]
            #print(type(dd))
            #print(dd[0])
            #print(dd[1])
            
            if dd[0] != dd[1]:
                n1 = set(list(fb.neighbors(dd[0])))
                #print(n1)
                #print(n2)
                n2 = set(list(fb.neighbors(dd[1])))
                intersection = list(n1.intersection(n2))
                #interseclist.append(intersection)
                ls = []
                #for e in range(len(intersection)):
                
                if len(intersection) == 1 :
                    ls.append(dd[0])
                    ls.append(dd[1])
                    ppe.append(ls)
             
#print(len(interseclist))
#interseclist
#print(len((set(interseclist))))
print(len(ppe))

## Post processing clean up!! 

# remove exact replications : eg [a,b] & [a,b] then only one is kept 
b_se = set(map(tuple,ppe))  #need to convert the inner lists to tuples so they are hashable
nodup = list(map(list,b_se))
print(len(nodup))

# works : if . [a,b] and [b, a] are in the list then only one either [a,b] or [b,a] is selected
fset = set(frozenset(x) for x in nodup)
lst = [list(x) for x in fset]
print(len(lst))
lst

# 
#to remove duplications: [a,a] should be discarded
lsst = []
for a in range(len(lst)):
    ll = lst[a]
    l = []
    if ll[0] != ll[1]:
        l.append(ll[0])
        l.append(ll[1])
        lsst.append(l)
print(len(lsst))

# lsst is thhe final list to use ! lsst give a list of edges taht are predicted to not get connected

In [ ]:
# roughly 130 lines of code